In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
import requests
import json
from bs4 import BeautifulSoup

In [5]:
from itertools import islice
def chunk(it, size):
    it = iter(it)
    return iter(lambda: tuple(islice(it, size)), ())

In [6]:
rootSite = "https://www.unstudio.com"
driver = webdriver.Chrome()
driver.get(rootSite+"/en/projects")

n_scrolls = 800
for j in range(0, n_scrolls):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(5)

anchorTag = (driver.find_elements(By.TAG_NAME, "a"))

In [7]:
projectsLink = []

for a in anchorTag:
    if ("/en/page/" in a.get_attribute("href")):
        projectsLink.append(a.get_attribute("href"))


In [8]:
projectsLink

['https://www.unstudio.com/en/page/3987/contact',
 'https://www.unstudio.com/en/page/7427/services',
 'https://www.unstudio.com/en/page/1006/cultural',
 'https://www.unstudio.com/en/page/1007/education',
 'https://www.unstudio.com/en/page/1008/exhibitions',
 'https://www.unstudio.com/en/page/14221/experience-design',
 'https://www.unstudio.com/en/page/16476/healthcare',
 'https://www.unstudio.com/en/page/1009/hotel',
 'https://www.unstudio.com/en/page/1010/industrial',
 'https://www.unstudio.com/en/page/1011/infrastructure',
 'https://www.unstudio.com/en/page/1012/interior',
 'https://www.unstudio.com/en/page/1022/living',
 'https://www.unstudio.com/en/page/1013/mixed-use',
 'https://www.unstudio.com/en/page/1014/museums',
 'https://www.unstudio.com/en/page/1015/offices',
 'https://www.unstudio.com/en/page/1016/pavilions',
 'https://www.unstudio.com/en/page/1017/products',
 'https://www.unstudio.com/en/page/1018/public-building',
 'https://www.unstudio.com/en/page/13619/refurbishment',

In [9]:
projectObjects = []
for link in projectsLink:
    driver.get(link)
    time.sleep(5)
    projectObject = {}
    projectObject["link"] = link
    try:
        projectTitle = driver.find_element(By.CLASS_NAME, "project-information__content").find_element(By.TAG_NAME, "h1").text
        projectObject["title"] = projectTitle
        print(projectTitle)
    except:
        pass
    try:
        city, country, year = driver.find_element(By.CLASS_NAME, "project-information__content").find_element(By.TAG_NAME, "h2").text.split(", ")
        projectObject["city"] = city
        projectObject["country"] = country
        projectObject["year"] = year
    except:
        pass
    try:
        datums = driver.find_element(By.CLASS_NAME, "project-information__aside").find_elements(By.TAG_NAME, "p")
        for datum in datums:
            sideData = (datum.text.split("\n "))[0]
            if sideData != "Share":
                rawFilterData = (sideData.split("\n"))
                cleanFilterData = [x for x in rawFilterData if x != '']
                try:
                    pairObjects = list(chunk(cleanFilterData, 2))
                    for pair in pairObjects:
                        projectObject[pair[0]] = pair[1]
                except:
                    pass
    except:
        pass
    projectObjects.append(projectObject)

Arnhem Central Masterplan
Mercedes-Benz Museum
Cockle Bay
Central Place
Castle Towers
The Bridge
The Bridge
Administration building for the European Stability Mechanism and the Luxembourg State
Transform Exhibition - Zumtobel Group’s Light Forum
CFL Luxembourg
Shenzhen Bay Headquarters Tower C
NION
Next Hotel
FlySolo Rehabilitation Medical Centre
Palas Residential
Future Perfect
Loft 02
Loft 01
Loft 02
Loft 01
Marina Residential Tower
Belsenpark
Marina Galleria
Vouliagmenis Mall and Tower - Ellinikon Quarters
Headquarters YG Entertainment
Shanghai Jiuguang Center
Eastland Melbourne Masterplan
5YN3RGY Media Harbour
IT Headquarters Amsterdam
K31 Courtyard
ConfEx Park
The Coolest White Paint Featured at State of Extremes Exhibit
Human-centered Design for Real Estate: Focusing on Inhabitants’ Experience
Node+
SunRider
Benchlands
Samara Arena Masterplan
Sochi Waterfront Concept Masterplan
Chungnam Art Museum
Airport East Integrated Hub
Elysion Congress Centre
JetBrains Office Campus
Shenzhe

In [10]:
import json
with open("UNSProjectData.json", "w") as jsonFile:
    json.dump(projectObjects, jsonFile, indent=2)

In [9]:
import json
with open("UNSProjectData.json", "r") as jsonFile:
    loadedJason = json.load(jsonFile)

filteredProjects = []
for i in loadedJason:
    if len(i.keys()) > 1:
        filteredProjects.append(i)


In [12]:
import pandas as pd

df = pd.DataFrame(filteredProjects, columns=["title", "city", "country", "year", "Client", "Status", "Size", "Type", "link"])
df

,title,city,country,year,Client,Status,Size,Type,link
0,Arnhem Central Masterplan,Arnhem,The Netherlands,2015,NaN,Completed,NaN,NaN,https://www.unstudio.com/en/page/12109/arnhem-...
1,Mercedes-Benz Museum,Stuttgart,Germany,2001-2006,"DaimlerChrysler Immobilien, Berlin",Completed,NaN,NaN,https://www.unstudio.com/en/page/12482/mercede...
2,Cockle Bay,Sydney,Australia,2016-2019,GPT and AMP Capital,Competition,NaN,NaN,https://www.unstudio.com/en/page/16722/cockle-bay
3,Central Place,Sydney,Australia,2019,Dexus and Frasers Property,Competition,NaN,NaN,https://www.unstudio.com/en/page/16711/central...
4,Castle Towers,Sydney,Australia,2013-2017,QICGRE,NaN,NaN,NaN,https://www.unstudio.com/en/page/16694/castle-...
...,...,...,...,...,...,...,...,...,...
295,Piet Hein Tunnel,Amsterdam,Netherlands,1990-1997,"Stedelijk Beheer Amsterdam, Projectbureau Piet...",Completed,NaN,NaN,https://www.unstudio.com/en/page/12071/piet-he...
296,Boat terminal,Yokohama,Japan,1994,NaN,"Competition entry 1994, 2nd prize",NaN,NaN,https://www.unstudio.com/en/page/12056/boat-te...
297,Electricity substation (50/10 kV),Amersfoort,Netherlands,1989-1993,Regionale Energie Maatschappij Utrecht (REMU),Completed,NaN,NaN,https://www.unstudio.com/en/page/11974/electri...
298,Renovation and extension Rijksmuseum Twenthe,Enschede,Netherlands,1992-1996,"Rijksgebouwendienst, Den Haag",Completed,NaN,NaN,https://www.unstudio.com/en/page/11877/renovat...
